### Import Required Modules

In [50]:
import pandas as pd
from pandas.io.json import json_normalize
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.geometry import Geometry
from arcgis import GIS
import requests
import json
import time
import math
import datetime
import urllib.parse
import numpy as np

### Spire API Key

In [51]:
auth_token = "yzNObz76itGCSTOXEvx3JojZpfYFNrB8"

### AIS Query Parameters

In [52]:
API_endpoint = 'https://api.sense.spire.com/vessels'
output_format = 'json'
limit = 1000
waittime = 20 #in seconds
# Rotterdam
geoJSON = """{"type":"Polygon","coordinates":[[[3.9111328125000004,51.89513899196507],[4.211883544921875,51.89513899196507],[4.211883544921875,52.01531743663362],[3.9111328125000004,52.01531743663362],[3.9111328125000004,51.89513899196507]]]}"""
# Savannah
#geoJSON = """{"type":"Polygon","coordinates":[[[-81.31805419921875,31.836732676846065],[-80.76324462890625,31.836732676846065],[-80.76324462890625,32.29525895520317],[-81.31805419921875,32.29525895520317],[-81.31805419921875,31.836732676846065]]]}"""
# Philly
#geoJSON = """{"type":"Polygon","coordinates":[[[-75.21240234375,39.88036665897025],[-75.10030746459961,39.88036665897025],[-75.10030746459961,39.95541184288875],[-75.21240234375,39.95541184288875],[-75.21240234375,39.88036665897025]]]}"""

### Query AIS Data from SPIRE API

In [53]:
def queryAIS(now):
    #print('Start Querying SPIRE Data...')
    #now = datetime.datetime.now()
    newiso = datetime.datetime.isoformat(now)
    headers = {"Authorization": "Bearer {}".format(auth_token), 'Accept': 'application/{}'.format(output_format)}
    request = '{}/?updated_after={}&limit={}&last_known_position_within={}'.format(API_endpoint,newiso,limit,geoJSON) 
    response = requests.get(request, headers=headers)
    data = response.json()
    datajson = json.loads(response.text)
    return datajson
    #print('Done.')

### Create a Spatially Enabled DataFrame Using the ArcGIS API for Python

In [54]:
def getRotatedCoords(x,y,xc,yc,angle):
    x = x - xc  
    y = y - yc
    angle = angle * -1  
    angle = math.radians(angle)  
    xr = (x * math.cos(angle)) - (y * math.sin(angle)) + xc  
    yr = (x * math.sin(angle)) + (y * math.cos(angle)) + yc  
    return [xr, yr]   

In [55]:
def point_to_poly(row):
    #get data from the Series
    coords = row["last_known_position.geometry.coordinates"]
    rotation = row["last_known_position.heading"]
    if rotation > 360:
        rotation = 0
    if row["length"] > 0:
        shipLength = row["length"]
    else:
        shipLength = 10
    if row["width"] > 0:
        shipWidth = row["width"]
    else:
        shipWidth = 4
    x_lon = coords[0]
    y_lat = coords[1]
    
    #Convert GCS to Web Mercator Coordinates
    if abs(x_lon) <= 180 and abs(y_lat) < 90:          
        num = x_lon * 0.017453292519943295         
        x = 6378137.0 * num         
        a = y_lat * 0.017453292519943295          
        x_mercator = x         
        y_mercator = 3189068.5 * math.log((1.0 + math.sin(a)) / (1.0 - math.sin(a)))

        polygon = [
            [x_mercator - (shipWidth * 0.5), y_mercator - (shipLength * 0.5)],
            [x_mercator - (shipWidth * 0.5), y_mercator + (shipLength * 0.350)],
            [x_mercator - (shipWidth * 0.431), y_mercator + (shipLength * 0.392)],
            [x_mercator - (shipWidth * 0.357), y_mercator + (shipLength * 0.427)],            
            [x_mercator - (shipWidth * 0.281), y_mercator + (shipLength * 0.455)],
            [x_mercator - (shipWidth * 0.203), y_mercator + (shipLength * 0.478)],
            [x_mercator - (shipWidth * 0.123), y_mercator + (shipLength * 0.492)],     
            [x_mercator - (shipWidth * 0.088), y_mercator + (shipLength * 0.495)],
            [x_mercator,y_mercator + (shipLength * 0.5)],
            [x_mercator,y_mercator + (shipLength * 0.5)],
            [x_mercator + (shipWidth * 0.088), y_mercator + (shipLength * 0.495)],
            [x_mercator + (shipWidth * 0.123), y_mercator + (shipLength * 0.492)],
            [x_mercator + (shipWidth * 0.203), y_mercator + (shipLength * 0.478)],
            [x_mercator + (shipWidth * 0.281), y_mercator + (shipLength * 0.455)],
            [x_mercator + (shipWidth * 0.357), y_mercator + (shipLength * 0.427)],
            [x_mercator + (shipWidth * 0.431), y_mercator + (shipLength * 0.392)],
            [x_mercator + (shipWidth * 0.5), y_mercator + (shipLength * 0.350)],                                                
            [x_mercator + (shipWidth * 0.5), y_mercator - (shipLength * 0.5)],
            [x_mercator - (shipWidth * 0.5), y_mercator - (shipLength * 0.5)],
        ]        
        
        #Get Rotated Coordinates
        rotatedPoly = []
        for vertex in polygon:
            rotatedPoly.append(getRotatedCoords(vertex[0],vertex[1],x_mercator,y_mercator,rotation))  
        
        return Geometry({"rings": [rotatedPoly],
                "spatialReference": {"wkid": 3857}})
    else:
        return None

In [56]:
def createSpatialDataFrame(datajson):
    df = pd.DataFrame(data=json_normalize(datajson['data']))
    df['SHAPE']=df.apply(point_to_poly, axis=1)
    df.spatial.set_geometry('SHAPE')
    return df

### Create the Map

In [57]:
aisMap = GIS().map('Europoort')
aisMap.basemap = 'satellite'
aisMap

MapView(layout=Layout(height='400px', width='100%'))

In [58]:
for i in range(1):
    shipLocations = queryAIS(datetime.datetime.now())
    df = createSpatialDataFrame(shipLocations)
    df.spatial.plot(map_widget=aisMap,
                           cmap = [255,255,255,200],
                           symbol_type='simple',
                           symbol_style='s',
                           outline_style='s',
                           outline_color=[0,0,0,255],
                           line_width=0.5)
        
    print('Total AIS responses: {}'.format(len(df)))
    time.sleep(waittime)

/Users/josh6643/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)
/Users/josh6643/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)


Total AIS responses: 238


In [ ]:
df

In [40]:
# interrogate the SeDF itself
q = df.SHAPE > 0

# get the data
df[q].head()

TypeError: '>' not supported between instances of 'Polygon' and 'int'